# [Deep Learning](https://www.cc.gatech.edu/~hays/compvision/proj6/)

## Setup

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import os.path as osp
import matplotlib.pyplot as plt
from utils import *
import student_code as sc
import torchvision.models as models
data_path = osp.join('../data', '15SceneData')
num_classes = 15

# If you have a good Nvidia GPU with an appropriate environment, 
# try setting the use_GPU flag to True (the environment provided does
# not support GPUs and we will not provide any support for GPU
# computation in this project). Please note that 
# we will evaluate your implementations only using CPU mode so even if
# you use a GPU, make sure your code runs in the CPU mode with the
# environment we provided. 
use_GPU = False
if use_GPU:
    from utils_gpu import *

To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

This project has two main parts. In Part 1, you will train a deep network from scratch. In Part 2, you will "fine-tune" a trained network. 

## Part 0. Warm up! Training a Deep Network from Scratch

In [2]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

You do not need to code anything for this part. You will simply run the code we provided, but we want you to report the result you got. This section will also familiarize you with the steps of training a deep network from scratch. 

In [3]:
# Training parameters.
input_size = (64, 64)
RGB = False  
base_lr = 1e-3  # may try a smaller lr if not using batch norm
weight_decay = 5e-4
momentum = 0.9

We will first create our datasets, by calling the create_datasets function from student_code. This function returns a separate dataset loader for each split of the dataset (training and testing/validation). Each dataloader is used to load the datasets after appling some pre-processing transforms. In Part 1, you will be asked to add a few more pre-processing transforms to the dataloaders by modifying this function.

In [4]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 30
Batch 20 / 30
Done, mean = 
[0.45579668]
std = 
[0.23624939]
Computing pixel mean and stdev...
Batch 0 / 60
Batch 20 / 60
Batch 40 / 60
Done, mean = 
[0.45517009]
std = 
[0.2350788]


Now we will create our network model using the SimpleNet class from student_code. The implementation provided in the SimpleNet class gives you a basic network. In Part 1, you will be asked to add a few more layers to this network. 

In [5]:
# Create the network model.
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

SimpleNet(
  (features): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): ReLU()
    (12): Dropout(p=0.75)
  )
  (classifier): Sequential(
    (0): Conv2d(32, 15, kernel_size=(8, 8), stride=(1, 1), bias=False)
  )
)

Next we will create the loss function and the optimizer. 

In [6]:
# Create the loss function.
# see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
loss_function = nn.CrossEntropyLoss()

In [7]:
# Create the optimizer and a learning rate scheduler
optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
# Currently a simple step scheduler.
# See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
# and how to use them
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

Finally we are ready to train our network! We will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [ ]:
# train the network!
params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 603, in urlopen
    chunked=chunked)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 355, in _make_request
    conn.request(method, url, **httplib_request

Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
---------------------------------------
Experiment: part1
resume_optim: True
num_workers: 4
experiment: part1
val_freq: 1
checkpoint_file: None
shuffle: True
print_freq: 100
n_epochs: 100
do_val: True
batch_size: 50
---------------------------------------
part1 Epoch 0 / 100
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 603, in urlopen
    chunked=chunked)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 355, in _make_request
    conn.request(method, url, **httplib_request

train part1: batch 0/29, loss 5.310, top-1 accuracy 6.000, top-5 accuracy 38.000
train part1: loss 5.151722
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 603, in urlopen
    chunked=chunked)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 355, in _make_request
    conn.request(method, url, **httplib_request

...

train part1: batch 0/29, loss 1.177, top-1 accuracy 56.000, top-5 accuracy 90.000
train part1: loss 1.145225
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 603, in urlopen
    chunked=chunked)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 355, in _make_request
    conn.request(method, url, **httplib_request

val part1: batch 0/59, loss 1.740, top-1 accuracy 38.000, top-5 accuracy 88.000
val part1: loss 1.275286
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Checkpoint saved
part1 Epoch 71 / 100
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 603, in urlopen
    chunked=chunked)
  File "/Users/diwakar/anaconda3/envs/cs4476p4/lib/python3.5/site-packages/urllib3/connectionpool.py", line 355, in _make_request
    conn.request(method, url, **httplib_request

train part1: batch 0/29, loss 1.342, top-1 accuracy 50.000, top-5 accuracy 94.000


Expect this code to take around 5 minutes on CPU or 3 minutes on GPU. Now you are ready to actually modify the functions we used to train our model. Before you move on, make sure to record the accuracy of your network from Part 0, and report it in your write up. 

## Part 1: Modifying the Dataloaders and the Simple Network create_datasets

In [ ]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

In [ ]:
# Training parameters.
input_size = (64, 64)
RGB = False  
base_lr = 1e-2  # may try a smaller lr if not using batch norm
weight_decay = 5e-4
momentum = 1.0

Now you will modify the create_datasets function from student_code. You will add random left-right mirroring and normalization to the transformations applied to the training dataset. You will also add normalization to the transformations applied to the testing dataset. 

In [ ]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Now you will modify SimpleNet by adding droppout, batch normalization, and additional convolution/maxpool/relu layers. You should achieve an accuracy of at least **50%**. Make sure your network passes this threshold--it is required for full credit on this section!

You can also use the following two blocks to determine the stucture of your network.

In [ ]:
# create the network model
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

In [ ]:
# Use this block to determine the kernel size of the conv2d layer in the classifier
# first, set the kernel size of that conv2d layer to 1, and run this block
# then, use that size of input to the classifier printed by this block to
# go back and update the kernel size of the conv2d layer in the classifier
# Finally, run this block again and verify that the network output size is a scalar
# Don't forget to re-run the block above every time you update the SimpleNet class!
from torch.autograd import Variable
data, _ = train_dataset[0]
s = data.size()
data = Variable(data.view(1, *s))
if use_GPU:
    data = data.cuda()
out = model(data)
print('Network output size is ', out.size())

Next we will create the loss function and the optimizer. You do not have to modify the custom_part1_trainer in student_code if you use the same loss_function, optimizer, scheduler and parameters (n_epoch, batch_size etc.) as provided in this notebook to hit the required threshold of 50% accuracy. If you changed any of these values, it is important that you modify this function in student_code since we will not be using the notebook you submit to evaluate. 

In [ ]:
# Set up the trainer. You can modify custom_part1_trainer in
# student_copy.py if you want to try different learning settings.
custom_part1_trainer = sc.custom_part1_trainer(model)

if custom_part1_trainer is None:
    # Create the loss function.
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Create the optimizer and a learning rate scheduler.
    optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
    # Currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

    params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
    
else:
    if 'loss_function' in custom_part1_trainer:
        loss_function = custom_part1_trainer['loss_function']
    if 'optimizer' in custom_part1_trainer:
        optimizer = custom_part1_trainer['optimizer']
    if 'lr_scheduler' in custom_part1_trainer:
        lr_scheduler = custom_part1_trainer['lr_scheduler']
    if 'params' in custom_part1_trainer:
        params = custom_part1_trainer['params']

We are ready to train our network! As before, we will start a local server to see the training progress of our network (if you server is already running, you should not start another one). Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [ ]:
# Train the network!
t = time.time()
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))
print(time.time() - t)

Make sure you get at least 50% accuracy in this section! If you tried different settings than the ones provided to get 50%, you should modify custom_part1_trainer in student code to return a dictionary with your changed settings. 

## Part 2. Fine-Tuning a Pre-Trained Network

In [ ]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

Training a network from scratch takes a lof of time. Instead of training from scratch, we can take a pre-trained model and fine tune it for our purposes. This is the goal of Part 2--you will train a pre-trained network, and achieve at least 80% accuracy. 

In [ ]:
# training parameters
input_size = (224, 224)
RGB = True
base_lr = 1e-3
weight_decay = 5e-4
momentum = 0.9
backprop_depth = 3

In [ ]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Following block loads a pretrained AlexNet.

In [ ]:
# Create the network model.
from torchvision.models import alexnet
model_alex = alexnet(pretrained=True, progress=True)
print(model_alex)

Now, you modify create_part2_model from student code in order to fine-tune AlexNet. As you can see in the docs (https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py) and in the model printout above, AlexNet has 2 parts: 'features', which constists of conv layers that extract feature maps from the image, and 'classifier' which consists of FC layers that classify the features. We want to replace the last Linear layer in model.classifier. 

In [ ]:
model = sc.create_part2_model(model_alex, num_classes)
if use_GPU:
    model = model.cuda()
print(model)

Next we will create the loss function and the optimizer. Just as with part 1, if you modify any of the setttings to hit the required accuracy, you must modify custom_part2_trainer function to return a dictionary containing your changes. 

In [ ]:
# Set up the trainer. You can modify custom_part2_trainer in
# student_copy.py if you want to try different learning settings.
custom_part2_trainer = sc.custom_part2_trainer(model)

if custom_part2_trainer is None:
    # Create the loss function
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Since we do not want to optimize the whole network, we must extract a list of parameters of interest that will be
    # optimized by the optimizer.
    params_to_optimize = []

    # List of modules in the network
    mods = list(model.features.children()) + list(model.classifier.children())

    # Extract parameters from the last `backprop_depth` modules in the network and collect them in
    # the params_to_optimize list.
    for m in mods[::-1][:backprop_depth]:
        params_to_optimize.extend(list(m.parameters()))

    # Construct the optimizer    
    optimizer = optim.SGD(params=params_to_optimize, lr=base_lr, weight_decay=weight_decay, momentum=momentum)

    # Create a scheduler, currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    params = {'n_epochs': 4, 'batch_size': 10, 'experiment': 'part2'} 
    
else:
    if 'loss_function' in custom_part2_trainer:
        loss_function = custom_part2_trainer['loss_function']
    if 'optimizer' in custom_part2_trainer:
        optimizer = custom_part2_trainer['optimizer']
    if 'lr_scheduler' in custom_part2_trainer:
        lr_scheduler = custom_part2_trainer['lr_scheduler']
    if 'params' in custom_part2_trainer:
        params = custom_part2_trainer['params']

We are ready to fine tune our network! Just like before, we will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 2 on the top left bar where is says Environment (only select Part 2, do not check main or Part 1).

In [ ]:
# Train the network!
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

Expect this code to take around 10 minutes on CPU or 30 seconds on GPU. You should hit 80% accuracy. 

## Write Up

### Part 1

#### 1. How did you do color jittering and nomalization? Why are they helpful?

1. Jittering: For scenes, a certain transformation like scaling, randomly changing brightness, contrasts, hue, etc., zooming or rotating in a random amount, mirroring can cause the label associated with the image to be the same. This image transformation is called jittering. The color jittering is implemented using function torchvision.transforms.ColorJitter() followed by mirroring which is implemented using the function torchvision.transforms.RandomHorizontalFlip(p=0.5), where p=0.5 means half of the images will be mirrored. This is helpful as it causes the network not to overfit on the original training images. Also, we can synthetically increase our amount of training data by left-right mirroring training images during the learning process. 

2. Normalization: Normalization makes classification loss less sensitive to changes in the weights and is easier to optomize. The mean and the standard deviation from the training images were computed. Then normalization is implemented using the function torchvision.transforms.Normalize(train_mean, train_std)). This transform normalizes each channel of the input image i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]. 

#### 2. How did you build your model? (what are sequential network, feature extractor, and classifier?)

#### 3. Talk more about the operations (Conv2d, Maxpool, Relu etc.) you used. Explain why they are important for deep model.

1. Conv2d: It applies a 2D convolution over an input image composed of several channels. More specifically, the convolution layer computes the output of neurons that are connected to local regions in the input, each computing a dot product between their weights and a small region they are connected to in the input volume. In particular, the transformations performed are a function of not only the activations in the input volume but also of the parameters (the weights and biases of the neurons). 

2. ReLU: RELU layer applies an elementwise activation function, such as the max(0,x) thresholding at zero. This leaves the size of the volume unchanged. It greatly accelerates the convergence of stochastic gradient descent compared to the sigmoid/tanh functions. This can be attributed due to its linear, non-saturating form.

3. MaxPool: It performs a downsampling operation along the spatial dimensions (width, height), resulting in the downsampling of the image, to better capture global features. The pooling layer serves to progressively reduce the spatial size of the representation, to reduce the number of parameters, memory footprint and amount of computation in the network, and hence to also control overfitting. 

4. Dropout: It is a regularization, used to avoid overfitting. It turns off a certain percentage of activations, to avoid dependency on a few neurons learning most of the model. Although it slightly slows down the learning, the accuracy increase compensates for it. 

5. BatchNorm2d: Batch Normalization reduces internal covariate shift, and in doing so dramatically accelerates the training of deep neural nets. It accomplishes this via a normalization step that fixes the means and variances of layer inputs. It also has a beneficial effect on the gradient flow through the network, by reducing the dependence of gradients on the scale of the parameters or of their initial values. This allows us to use much higher learning rates without the risk of divergence. Furthermore, batch normalization regularizes the model and reduces the need for Dropout.


#### 4. How did you do data transformation? Also, disucss anything else you need to do to achieve require performance.

First, all the training images are resized to the input size of 64 using torchvision.transforms.resize() and torchvision.transforms.CenterCrop(). Then all the images were mirrored with a probability of 0.5 to avoid the network to overfit to the 1500 original training images. This is done by calling the function torchvision.transforms.RandomHorizontalFlip(p=0.5). After this, the whole dataset is normalized by subtracting the mean and dividing by the standard deviation of training images. This is implemented using the function torchvision.transforms.Normalize(mean, std).

To improve the performance, the weights of the convolution layers are initialized using Kaiming_normal. This is implemented using nnnn.init.kaiming_normal_(). Also, the optimizer used is optim.Adam() which uses the Adam algorithm. Also, to make the network deep, two additional convolution layers are added along with the max-pool layer and ReLU layer as well.

## Part 2. Fine-Tuning a Pre-Trained Network
#### What is the fine-tuning? What is the benifit of fine-tuing?

Fine-tuning is a scenario in which the final layer or more of an existing network is replaced with random weights, and the entire network is trained again with images and ground truth labels for the corresponding recognition task. The idea is to effectively treat the pre-trained deep network as a better initialization than the random weights used when training from scratch. 

When the training data is not enough to train a complex network from scratch (e.g. with the 15 scene database) fine-tuning work quite efficiently. Fine-tuning often exceeds the performance of hand-crafted features when used in a new domain.

Alexnet is a convolution neural network with 5 convolutional layers (with some ReLU and max pool) as the main feature layers, followed by 3 Fully Connected linear layers. I crossed the 80% threshold by discarding the last layer and then adding a layer to the classifier of AlexNet that transforms 4096 features (output of the first fully connected layer) to the 15 possible categories. The weights of the added layer are initialized using the method used in Part 1. After training the model, the accuracy improved to 84.992%.

In [ ]:
print('The loss and top-1 accuracy for Part 2')
image1 = cv2.imread('../code/part2_loss.png')
# image1 = load_image('../code/part1_loss.png')
plt.figure(); 
plt.imshow(image1)
print('Figure 1')
image1 = cv2.imread('../code/part2_acc.png')
plt.figure();
plt.imshow(image1)
print('Figure 2')

### Extra Credits








I optimized my part 1 network to achieve the maximum accuracy of 68%. In order to avoid overfitting, I've used dropout with probability of 0.75 before my last convolution layer. Consequently, the gap between the validation loss and training loss is very small after 100 epochs. 

In [ ]:
image1 = cv2.imread('../code/part1_loss.png')
# image1 = load_image('../code/part1_loss.png')
plt.figure(); 
plt.imshow(image1)
print('Figure 1')
image1 = cv2.imread('../code/part1_acc.png')
plt.figure();
plt.imshow(image1)
print('Figure 2')